In [6]:
!pip install bs4
!pip install geopy
!pip install folium==0.5.0

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=f6fdd76b9eb0bdcacddb4e745b4eb05f0dca9e9853e04c1f0144877add9a2b36
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [8]:
url='https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Vancouver'

In [9]:
wikidata = requests.get(url).text

create a beautiful soup object

In [10]:
soup = BeautifulSoup(wikidata,"html5lib")

In [12]:
neighborhoods = [] # empty dictionary

ul_list = soup.findAll('ul')
ul_index = -1
for index,ul in enumerate(ul_list):
    if('Arbutus_Greenway') in str(ul):
        ul_index = index
if (ul_index == -1):
    print("list not found")
else:
    print("the index of the neighborhood ul element is ", ul_index)

the index of the neighborhood ul element is  1


Data Cleansing. The table contains an unordered list, with each neighborhood identified by a href object within a list item and the descriptions contain a repeat of the neighborhood name.  Parse the soup object to get the name and descript. Use python to strip out the redundant name.  

In [14]:
 
rows = ul_list[ul_index]

for li in rows.findAll('li'):
    cell = {}
    
    for link in li.find('a'):
        cell['Name'] = link.lstrip().rstrip()
    
    basedesc = str(li.text)
    strindex = basedesc.find('-')
    if (strindex > 0):
        strstart = strindex + 1
    
    strend = len(basedesc) - strstart
    cell['Description']= basedesc[strstart:strend].lstrip()
    neighborhoods.append(cell)
       

df = pd.DataFrame(neighborhoods)
df.replace("Arbutus Ridge", 
           "Arbutus-Ridge", 
           inplace=True)
df    

,Name,Description
0,Arbutus-Ridge,Located in the middle of Vancouver's west side...
1,Downtown,"The Central business district of Vancouver, co..."
2,Dunbar-Southlands,Southlands - An affluent neighbourhood on the ...
3,Fairview,Contains the popular attractions of Granville ...
4,Grandview-Woodland,Woodland - A mature neighbourhood in east Vanc...
5,Hastings-Sunrise,Sunrise - One of Vancouver's oldest neighbourh...
6,Kensington-Cedar Cottage,Cedar Cottage - One of the most multicultural ...
7,Kerrisdale,A primarily residential neighbourhood bisected...
8,Killarney,Located in the southeastern corner of the city...
9,Kitsilano,"Located along the south end of English Bay, th..."


Confirm the shape of the list of neighborhoods to make sure we got all 22.

In [15]:
df.shape

(22, 2)

Again using BeautifulSoup to scrape the data, get the associated latitude and longitude of each neighborhood from the City of Vancouver Open Data Portal.

In [16]:
odpurl='https://opendata.vancouver.ca/explore/dataset/local-area-boundary/download/?format=json&timezone=America/Los_Angeles&lang=en'
odpdata = requests.get(odpurl).json()


The relevant parts of the JSON are the id, name and geo_point_2d, which contains the lat/long coordinates.

In [17]:
dfdtl = json_normalize(odpdata)
dfdtl

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,datasetid,recordid,record_timestamp,fields.mapid,fields.geom.type,fields.geom.coordinates,fields.name,fields.geo_point_2d,geometry.type,geometry.coordinates
0,local-area-boundary,b5f389ecdc26e66765eb7e4ddb2b26f324791f38,2019-03-14T04:02:27.283-07:00,AR,Polygon,"[[[-123.15259552001953, 49.25722503661694], [-...",Arbutus-Ridge,"[49.2468049108, -123.161669238]",Point,"[-123.161669238, 49.2468049108]"
1,local-area-boundary,f422c2100e9cb31018314a144d02519705da7e71,2019-03-14T04:02:27.283-07:00,CBD,Polygon,"[[[-123.11226654052733, 49.29016494750564], [-...",Downtown,"[49.2807470711, -123.116567008]",Point,"[-123.116567008, 49.2807470711]"
2,local-area-boundary,03187cbe941218efd0347c39aaeaeb00a16c2980,2019-03-14T04:02:27.283-07:00,FAIR,Polygon,"[[[-123.14598846435547, 49.25712203979079], [-...",Fairview,"[49.2645404871, -123.131048865]",Point,"[-123.131048865, 49.2645404871]"
3,local-area-boundary,2024a0d84997f4740ce6e96a94e0d3d1b7d00b87,2019-03-14T04:02:27.283-07:00,GW,Polygon,"[[[-123.07701873779298, 49.29024887084548], [-...",Grandview-Woodland,"[49.2764396102, -123.066728221]",Point,"[-123.066728221, 49.2764396102]"
4,local-area-boundary,8e1445b0adfd38e2ace35525518f475dd5138cd6,2019-03-14T04:02:27.283-07:00,HS,Polygon,"[[[-123.05648803710938, 49.293487548824], [-12...",Hastings-Sunrise,"[49.277934053, -123.040269923]",Point,"[-123.040269923, 49.277934053]"
5,local-area-boundary,611d9ca697724e47f1e0997f5e07e4ec87a8e0d2,2019-03-14T04:02:27.283-07:00,MARP,Polygon,"[[[-123.10696411132812, 49.20415878295483], [-...",Marpole,"[49.2102074666, -123.12838166]",Point,"[-123.12838166, 49.2102074666]"
6,local-area-boundary,d167678fba0e23f8a82e4d48159cb06ed8aad571,2019-03-14T04:02:27.283-07:00,RP,Polygon,"[[[-123.10562133789061, 49.23311614989819], [-...",Riley Park,"[49.2447664796, -123.103146806]",Point,"[-123.103146806, 49.2447664796]"
7,local-area-boundary,44d12027e568fbf55be372856a07e2dda20ce1b9,2019-03-14T04:02:27.283-07:00,SHAU,Polygon,"[[[-123.15527343749999, 49.2345237731892], [-1...",Shaughnessy,"[49.2456809771, -123.139760167]",Point,"[-123.139760167, 49.2456809771]"
8,local-area-boundary,58b2b8fdd5e48ea00c79f0a33a90ec99bf98372a,2019-03-14T04:02:27.283-07:00,STR,Polygon,"[[[-123.0992889404297, 49.28927230834548], [-1...",Strathcona,"[49.2782195786, -123.088235281]",Point,"[-123.088235281, 49.2782195786]"
9,local-area-boundary,7245d84262949196150423f3a7f69adac8ca271b,2019-03-14T04:02:27.283-07:00,WE,Polygon,"[[[-123.13768005371092, 49.27531814574782], [-...",West End,"[49.2850111894, -123.135437653]",Point,"[-123.135437653, 49.2850111894]"


key colunms are fields.mapid, fields.name, fields.geometry_coordinates - we will drop the rest

In [18]:
flds=dfdtl.drop(['datasetid','recordid','record_timestamp','fields.geom.type','fields.geom.coordinates','geometry.type','geometry.coordinates'], axis=1)

flds

,fields.mapid,fields.name,fields.geo_point_2d
0,AR,Arbutus-Ridge,"[49.2468049108, -123.161669238]"
1,CBD,Downtown,"[49.2807470711, -123.116567008]"
2,FAIR,Fairview,"[49.2645404871, -123.131048865]"
3,GW,Grandview-Woodland,"[49.2764396102, -123.066728221]"
4,HS,Hastings-Sunrise,"[49.277934053, -123.040269923]"
5,MARP,Marpole,"[49.2102074666, -123.12838166]"
6,RP,Riley Park,"[49.2447664796, -123.103146806]"
7,SHAU,Shaughnessy,"[49.2456809771, -123.139760167]"
8,STR,Strathcona,"[49.2782195786, -123.088235281]"
9,WE,West End,"[49.2850111894, -123.135437653]"


split the lat/long into separate columns

In [19]:
lat = []
lon = []

for row in flds['fields.geo_point_2d']:
    lat.append(row[0])
    lon.append(row[1])
flds['latitude'] = lat
flds['longitude']= lon

flds

,fields.mapid,fields.name,fields.geo_point_2d,latitude,longitude
0,AR,Arbutus-Ridge,"[49.2468049108, -123.161669238]",49.246805,-123.161669
1,CBD,Downtown,"[49.2807470711, -123.116567008]",49.280747,-123.116567
2,FAIR,Fairview,"[49.2645404871, -123.131048865]",49.264540,-123.131049
3,GW,Grandview-Woodland,"[49.2764396102, -123.066728221]",49.276440,-123.066728
4,HS,Hastings-Sunrise,"[49.277934053, -123.040269923]",49.277934,-123.040270
5,MARP,Marpole,"[49.2102074666, -123.12838166]",49.210207,-123.128382
6,RP,Riley Park,"[49.2447664796, -123.103146806]",49.244766,-123.103147
7,SHAU,Shaughnessy,"[49.2456809771, -123.139760167]",49.245681,-123.139760
8,STR,Strathcona,"[49.2782195786, -123.088235281]",49.278220,-123.088235
9,WE,West End,"[49.2850111894, -123.135437653]",49.285011,-123.135438


Drop the geo_pont_2d column, rename the columns and join the geopoints with the neighborhood wiki data to add the descriptions

In [20]:
flds_geopoints = flds.drop(['fields.geo_point_2d'],axis=1)
flds_geopoints.columns=['Mapid','Name','Latitude','Longitude']
flds_geopoints

neighborhoods = pd.merge(flds_geopoints,df, on='Name')
neighborhoods

,Mapid,Name,Latitude,Longitude,Description
0,AR,Arbutus-Ridge,49.246805,-123.161669,Located in the middle of Vancouver's west side...
1,CBD,Downtown,49.280747,-123.116567,"The Central business district of Vancouver, co..."
2,FAIR,Fairview,49.264540,-123.131049,Contains the popular attractions of Granville ...
3,GW,Grandview-Woodland,49.276440,-123.066728,Woodland - A mature neighbourhood in east Vanc...
4,HS,Hastings-Sunrise,49.277934,-123.040270,Sunrise - One of Vancouver's oldest neighbourh...
5,MARP,Marpole,49.210207,-123.128382,The neighbourhood contains a diverse range of ...
6,RP,Riley Park,49.244766,-123.103147,Located in the centre of Vancouver and contain...
7,SHAU,Shaughnessy,49.245681,-123.139760,An affluent and mostly residential neighbourho...
8,STR,Strathcona,49.278220,-123.088235,"One of the city's oldest neighbourhoods, conta..."
9,WE,West End,49.285011,-123.135438,It is home to the city's historical gay commun...
